Connect to db (remove in later version that works with production database)

In [ ]:
import datajoint as dj
import os
from pathlib import Path

# set dirs
base_dir = Path('/hdd/dj') # change this to your desired directory
if (base_dir).exists() is False:
    os.mkdir(base_dir)
raw_dir = base_dir / 'raw'
if (raw_dir).exists() is False:
    os.mkdir(raw_dir)
analysis_dir = base_dir / 'analysis'
if (analysis_dir).exists() is False:
    os.mkdir(analysis_dir)
tmp_dir = base_dir / 'tmp'
if (tmp_dir).exists() is False:
    os.mkdir(tmp_dir)

# set dj config
dj.config['database.host'] = 'localhost'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'tutorial'
dj.config['database.port'] = 3306
dj.config['stores'] = {
  'raw': {
    'protocol': 'file',
    'location': str(raw_dir),
    'stage': str(raw_dir)
  },
  'analysis': {
    'protocol': 'file',
    'location': str(analysis_dir),
    'stage': str(analysis_dir)
  }
}
dj.config["enable_python_native_blobs"] = True


# set env vars
os.environ['SPYGLASS_BASE_DIR'] = str(base_dir)
os.environ['SPYGLASS_TEMP_DIR'] = str(tmp_dir)
os.environ['KACHERY_CLOUD_DIR'] = '/hdd/dj/.kachery-cloud'
os.environ['KACHERY_ZONE'] = "franklab.default"
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'


%load_ext autoreload
%autoreload 2

import

In [ ]:
import spyglass as sg
import spyglass.common as sgc
import spyglass.spikesorting.v1 as sgs
import spyglass.data_import as sgi

insert LabMember and Session

In [ ]:
nwb_file_name = "wilbur20210326.nwb"
nwb_file_name2 = "wilbur20210326_.nwb"

In [ ]:
sgc.LabMember.insert_from_nwbfile(nwb_file_name)

In [ ]:
sgi.insert_sessions(nwb_file_name)

In [ ]:
sgc.Session()

insert SortGroup

In [ ]:
sgs.SortGroup.set_group_by_shank(nwb_file_name=nwb_file_name2)

insert SpikeSortingRecordingSelection. use `insert_selection` method. this automatically generates a unique recording id

In [ ]:
key = {"nwb_file_name" : nwb_file_name2,
       "sort_group_id" : 0,
       "interval_list_name" : "03_r1",
       "preproc_param_name" : "default",
       "team_name" : "Alison Comrie"}

In [ ]:
sgs.SpikeSortingRecordingSelection.insert_selection(key)

In [ ]:
sgs.SpikeSortingRecordingSelection()

preprocess recording (filtering and referencing)

In [ ]:
sgs.SpikeSortingRecording.populate()

In [ ]:
sgs.SpikeSortingRecording()

In [ ]:
key = (sgs.SpikeSortingRecordingSelection & {"nwb_file_name":nwb_file_name2}).fetch1()

insert ArtifactDetectionSelection

In [ ]:
sgs.ArtifactDetectionSelection.insert_selection({'recording_id':key['recording_id'],
                                                 'artifact_param_name':'default'})

detect artifact; note the output is stored in IntervalList

In [ ]:
sgs.ArtifactDetection.populate()

In [ ]:
sgs.ArtifactDetection()

insert SpikeSortingSelection. again use `insert_selection` method

In [ ]:
key = {
        "recording_id":key['recording_id'],
        "sorter":"mountainsort4",
        "sorter_param_name": "franklab_tetrode_hippocampus_30KHz",
        "nwb_file_name":nwb_file_name2,
        "interval_list_name":str((sgs.ArtifactDetectionSelection & {'recording_id':key['recording_id']}).fetch1("artifact_id"))
    }

In [ ]:
sgs.SpikeSortingSelection()

In [ ]:
sgs.SpikeSortingSelection.insert_selection(key)

In [ ]:
sgs.SpikeSortingSelection()

run spike sorting

In [ ]:
sgs.SpikeSorting.populate()

In [ ]:
sgs.SpikeSorting()

we have two main ways of curating spike sorting: by computing quality metrics and applying threshold; and manually applying curation labels. to do so, we first insert CurationV1. use `insert_curation` method.

In [ ]:
sgs.CurationV1.insert_curation(sorting_id=(sgs.SpikeSortingSelection & {'recording_id':key['recording_id']}).fetch1("sorting_id"),
                               description="testing sort")

In [ ]:
sgs.CurationV1()

we will first do an automatic curation based on quality metrics

In [ ]:
key = {
    "sorting_id":(sgs.SpikeSortingSelection & {'recording_id':key['recording_id']}).fetch1("sorting_id"),
    "curation_id":0,
    "waveform_param_name":"default_not_whitened",
    "metric_param_name":"franklab_default",
    "metric_curation_param_name":"default"
}

In [ ]:
sgs.MetricCurationSelection.insert_selection(key)

In [ ]:
sgs.MetricCurationSelection()

In [ ]:
sgs.MetricCuration.populate()

In [ ]:
sgs.MetricCuration()

to do another round of curation, fetch the relevant info and insert back into CurationV1 using `insert_curation`

In [ ]:
key = {"metric_curation_id":(sgs.MetricCurationSelection & {'sorting_id':key['sorting_id']}).fetch1("metric_curation_id")}

In [ ]:
labels = sgs.MetricCuration.get_labels(key)

In [ ]:
merge_groups = sgs.MetricCuration.get_merge_groups(key)

In [ ]:
metrics = sgs.MetricCuration.get_metrics(key)

In [ ]:
sgs.CurationV1.insert_curation(sorting_id=(sgs.MetricCurationSelection & {'metric_curation_id':key['metric_curation_id']}).fetch1("sorting_id"),
                               parent_curation_id=0,
                               labels=labels,
                               merge_groups= merge_groups,
                               metrics=metrics,
                               description="after metric curation")

In [ ]:
sgs.CurationV1()

next we will do manual curation. this is done with figurl. to incorporate info from other stages of processing (e.g. metrics) we have to store that with kachery cloud and get curation uri referring to it. it can be done with `generate_curation_uri`.

In [ ]:
curation_uri = sgs.FigURLCurationSelection.generate_curation_uri({"sorting_id":(sgs.MetricCurationSelection & {'metric_curation_id':key['metric_curation_id']}).fetch1("sorting_id"),
                                                                  "curation_id":1})

In [ ]:
key = {"sorting_id":(sgs.MetricCurationSelection & {'metric_curation_id':key['metric_curation_id']}).fetch1("sorting_id"),
       "curation_id":1,
       "curation_uri": curation_uri,
       "metrics_figurl":list(metrics.keys())}

In [ ]:
sgs.FigURLCurationSelection()

In [ ]:
sgs.FigURLCurationSelection.insert_selection(key)

In [ ]:
sgs.FigURLCurationSelection()

In [ ]:
sgs.FigURLCuration.populate()

In [ ]:
sgs.FigURLCuration()

or you can manually specify it if you already have a `curation.json`

In [ ]:
gh_curation_uri = "gh://LorenFrankLab/sorting-curations/main/khl02007/test/curation.json"

key = {"sorting_id" : key["sorting_id"],
       "curation_id" : 1,
       "curation_uri" : gh_curation_uri,
       "metrics_figurl" : []}

In [ ]:
sgs.FigURLCurationSelection.insert_selection(key)

In [ ]:
sgs.FigURLCuration.populate()

In [ ]:
sgs.FigURLCuration()

once you apply manual curation (curation labels and merge groups) you can store them as nwb by inserting another row in CurationV1. And then you can do more rounds of curation if you want.

In [ ]:
labels = sgs.FigURLCuration.get_labels(gh_curation_uri)

In [ ]:
merge_groups = sgs.FigURLCuration.get_merge_groups(gh_curation_uri)

In [ ]:
sgs.CurationV1.insert_curation(sorting_id=key["sorting_id"],
                               parent_curation_id=1,
                               labels=labels,
                               merge_groups= merge_groups,
                               metrics=metrics,
                               description="after figurl curation")

In [ ]:
sgs.CurationV1()

In [ ]:
from spyglass.spikesorting.merge import SpikeSortingOutput

In [ ]:
SpikeSortingOutput.insert_from_source(key, source="CurationV1")

In [ ]:
SpikeSortingOutput()

In [ ]:
SpikeSortingOutput.CurationV1()